In [1]:
import os
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.onnx
import onnx

from google.protobuf import json_format

# First we export a PyTorch model
filename = "model.proto"
fullpath = os.path.join("/tmp", filename)

dummy_input = Variable(torch.FloatTensor([[1,2,3,4,5], [-5,-4,-3, 2, 1]]))
model = torch.nn.Sequential(
          torch.nn.Linear(5, 2),
#           torch.nn.Dropout(p=0.9),
#           torch.nn.ReLU(),
#           torch.nn.Linear(4, 3),
#           torch.nn.Dropout(p=0.9),
#           torch.nn.ReLU(),
#           torch.nn.Linear(3, 2),
          torch.nn.Softmax(dim=1),
        )
for param in model.parameters():
    print(param.data)
print(model, "\n")
torch.onnx.export(model, dummy_input, fullpath)
print("model saved at: %s" % fullpath)



 0.0163 -0.1790 -0.4116  0.3248 -0.0546
-0.0376  0.2728  0.1537 -0.0951 -0.2583
[torch.FloatTensor of size 2x5]


 0.2224
 0.4053
[torch.FloatTensor of size 2]

Sequential(
  (0): Linear(in_features=5, out_features=2)
  (1): Softmax()
) 

model saved at: /tmp/model.proto


In [2]:
import syft
import syft.nn
import syft.onnx
from syft import FloatTensor

/Users/morgangiraud/.virtualenvs/openmined/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [3]:
# Load the PyTorch model
syft_dummy_input = FloatTensor([[1,2,3,4,5],[-5,-4,-3,2,1]], autograd=True)
syft_model = syft.onnx.load(fullpath)

syft_model.summary()
print(syft_model.parameters(), "\n")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
linear_2 (Linear)            (None, 2)                 12
_________________________________________________________________
softmax_3 (Softmax)          (dynamic)                 0
Total params: 12
Trainable params: 12
Non-trainable params: 0
_________________________________________________________________

[[[ 0.0162746  -0.03755596]
 [-0.1789756   0.2728011 ]
 [-0.4116018   0.1536942 ]
 [ 0.3247608  -0.09514675]
 [-0.05463052 -0.2583194 ]]
[syft.FloatTensor:3 grad:None size:5x2 c:[] p:[2] init:transpose]

	-----------creators-----------
	[syft.FloatTensor:2 grad:None size:2x5 c:[3] p:[] init:]
	------------------------------


, [[ 0.2223567  0.4053208]]
[syft.FloatTensor:4 grad:None size:1x2 c:[] p:[] init:]
] 



In [4]:
# Execute models
pytorch_output = model(dummy_input)
syft_output = syft_model(syft_dummy_input)
print(dummy_input, pytorch_output)
print(syft_dummy_input)
print(syft_output)

Variable containing:
 1  2  3  4  5
-5 -4 -3  2  1
[torch.FloatTensor of size 2x5]
 Variable containing:
 0.4924  0.5076
 0.9836  0.0164
[torch.FloatTensor of size 2x2]

   1.  2.  3.  4.  5. 
  -5. -4. -3.  2.  1.  
   0.4923756   0.5076244  
   0.9836072   0.01639285  


In [5]:
# Testing consistency between serialization and deserialization in OpenMined
syft.onnx.export(syft_model, syft_dummy_input, fullpath)
new_syft_model = syft.onnx.load(fullpath)

syft_output = syft_model(syft_dummy_input)
new_syft_output = new_syft_model(syft_dummy_input)

print(syft_output)
print(new_syft_output)

   0.4923756   0.5076244  
   0.9836072   0.01639285  
   0.4923756   0.5076244  
   0.9836072   0.01639285  
